In [1]:
#!pip install tensorflow --upgrade
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
from torch import nn
import torch
import pandas as pd
import os
import os.path
from PIL import Image
from torchvision import models
from tensorflow import summary
import datetime
import matplotlib.pyplot as plt
import numpy as np
import torchsummary
import datetime
import json
import cv2
import itertools
%load_ext tensorboard

In [0]:
from google.colab import drive
drive.mount('/content/drive') 

In [0]:
%%shell

# Install pycocotools
git clone https://github.com/cocodataset/cocoapi.git >/dev/null
cd cocoapi/PythonAPI
python setup.py build_ext install>
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [0]:
%%shell

cd vision
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [0]:
import transforms
class ZebraDataset(Dataset):
  def __init__(self, path, jfile, transform = None):
    self.transform = transform
    self.path = path
    self.data = []
    dataset_dicts = []
    json_file = os.path.join(path, jfile)
    with open(json_file) as f:
        imgs_anns = json.load(f)

    for _, v in imgs_anns.items():
        record = {}
        annos = v["regions"]
        if not annos:
          continue

        filename = os.path.join(path, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            #not anno["region_attributes"]
            classattr = anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = list(itertools.chain.from_iterable(poly))
            if not classattr:
                cls = 0
            else :
                cls = int(classattr["class"])

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "segmentation": [poly],
                "category_id": cls,

                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    self.data = dataset_dicts[1:]
    
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self,id):
    data = self.data[id]
    file = Image.open(os.path.join(self.path,data["file_name"])).convert('RGB')
    target={}
    boxes = []
    label=[]
    for ann in data["annotations"]:
      boxes.append(ann["bbox"])
      label.append(int(ann["category_id"]))
    
    #print(len(boxes),len(label))
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    image_id = torch.tensor([id])
    target["boxes"] = boxes
    target["lables"] = torch.as_tensor(label, dtype=torch.int32)
    target["image_id"] = image_id
    target["area"] = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
    target["iscrowd"] = torch.zeros(len(data["annotations"]))
    


    if self.transform is not None:
      file, target = self.transform(file,target)
    return file, target

In [0]:
root = "/content/drive/My Drive/Colab Notebooks/data/images/gray"
jfile = "set 1-125.json"
trans = transforms.Compose(
   [ transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])]
    )
z=ZebraDataset(root,jfile=jfile)

In [0]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 5  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [0]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [0]:
device = torch.device('cuda')
num_classes

In [0]:
 z = ZebraDataset(root,jfile,get_transform(train=True))
 data_loader = torch.utils.data.DataLoader(
        z, batch_size=2, shuffle=True, num_workers=4,
        )


In [0]:
import torch.optim
model = model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
z.__getitem__(5)

In [0]:
num_epochs = 16
import utils

#from engine import train_one_epoch, evaluate
for epoch in range(num_epochs):
  # train for one epoch, printing every 10 iterations
  train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
  # update the learning rate
  lr_scheduler.step()
  # evaluate on the test dataset
  #evaluate(model, data_loader_test, device=device)
  

In [0]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        tar={}
        tar = {k: v.to(device) for k, v in targets.items()}
        targets = tar
        print("loss", images.shape, type(tar), tar)
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

In [0]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', utils.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = utils.warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])
